# Valuing American Options by Simulation: A Simple Least-Square Approach

Implementation by Python

For an American option, the optimal strategy to exercise is to compare the immediate exercise value with the expected cash flows from continuing. Thus, The key to optimally exercising an American option is identifying the conditional expected value of continuation.


$ dS = \mu Sdt + \sigma SdZ $

The formula for calculating stock price at time t is:

$ S_{t_{i+1}} = S_{t_{i}} e^{(\mu - \frac{1}{2}\sigma^2)(t_{i+1}-t_{i})+\sigma \sqrt{t_{i+1}-t_{i}} Z_{i+1}} $,
where $Z \sim N(0, 1)$

In [248]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from scipy.stats import norm
from mpl_toolkits.mplot3d import Axes3D

In [249]:
'''    
parameter description:
S0: initial price
n: number of steps
t: starting time
T: terminating time
St: trajectory of price
a, b: the first and second moment of log Y
d: dividend yield
'''

def Geometric_Brownian_Motion_Trajectory( mu, sigma, S0, n, t, T ): 
    time = np.linspace(t, T, n + 1) 
    delta_time = time[1] - time[0] 
    St = np.zeros(n + 1)
    St[0] = S0
    z = np.random.standard_normal(n) 
    for i in range(n):
        St[i + 1] = St[i] * np.exp((mu - 1 / 2 * sigma ** 2) * delta_time + sigma * delta_time ** (1 / 2) * z[i])
    return St

def Geometric_Brownian_Motion_Jump( mu, sigma, d, S0, n, t, T, a, b, lam ):
        
    delta_t = (T - t) / n
    St = np.zeros(n + 1)
    X = np.zeros(n + 1)
    z = np.random.normal(size=(n + 1, 1))
    X[0] = np.log(S0)
    for i in range(1, n + 1):
        n = np.random.poisson(lam * delta_t)
        if n == 0:
            m =0 
        else:
            m = a * n + b * n ** 0.5 * np.random.normal()
        X[i] = X[i - 1] + (mu - d - 0.5 * sigma ** 2) * delta_t + sigma * delta_t ** 0.5 * z[i] + m
        St = np.exp(X)
    return St


In [276]:
def Valuation_by_Least_Square( r, sigma, S0, m, n, t, T ):
    
    # Create m paths of stock price with n steps of time by simulation
    St_GeoBro = np.zeros((m, n+1))
    for i in range(m):
        St_GeoBro[i, :] = Geometric_Brownian_Motion_Trajectory( r, sigma, S0, n, t, T )

    # Payoff is a matrix of the amount of cash flow at each step if immediately exercising the option,
    # which is only for convenience of calculation in the following procedures 
    Payoff = np.maximum( K - St_GeoBro, 0 )
        
    # Cash_Flow is a matrix similar to Payoff, 
    # which is updated by doing regression and deciding whether to exercise immediately
    Cash_Flow = np.maximum( K - St_GeoBro, 0 )

    # Calculate the conditional expected value of continuation
    # 1. regressing (Y = the discounted payoff at time t_i+1) against (X = the stock price, whose option is in the money at time t_i) and X^2
    # 2. predict the expected conditional value at time t_i by substituing X and X^2 into the regression formula
    # 3. compare the expected conditional value with the immediate value
    # 4. if the immediate value is greater, exercise immediately

    for i in range(n-1):
    
        # X is the payoff if exercise in the money at time t_i
        X = ( Payoff[:, n-1-i] )[ Payoff[:, n-1-i] > 0 ]

        # Y is the discounted payoff at time t_i+1, related to X
        Y = ( Payoff[:, n-i] )[ Payoff[:, n-1-i] > 0 ] * np.exp( -r * 1/n * (i+1) )

        # L0, L1, L2 are basis functions of X
        # combine them into a single matrix for following regression 
        X = X.reshape(np.size(X),1)
        L0 = np.exp( -X/2 )
        L1 = L0 * ( 1 - X )
        L2 = L0 * ( 1 - 2*X + X**2/2 )
        XX = np.hstack((L0, L1, L2))

        # regress Y ~ intercept + a * X + b * X2
        reg = LinearRegression().fit(XX, Y)
        # calculate the predicted value of Y (i.e. the conditional expected value of continuation)
        Y_predict = reg.predict(XX)
        Y_predict = Y_predict.reshape(1,np.size(Y_predict))

        # compare the immediate exercise value with the conditional expected value of continuation
        # and decide whether to exercise immediately
        exercise = ( Y_predict < Payoff[ Payoff[:, n-1-i] > 0, n-1-i ] ) * Payoff[ Payoff[:, n-1-i] > 0, n-1-i ]

        # substitue those values decided to exercise immediately into the Cash_Flow matrix
        # and set the continuing values of Cash_Flow to zero, as the option is exercised obly once
        Cash_Flow[ Payoff[:, n-1-i] > 0, n-1-i ] = exercise
        Cash_Flow[ Payoff[:, n-1-i] > 0, n-i: ] = 0
    
    # calculate the discounted factor matrix
    df = np.ones( np.shape( Cash_Flow[:, 1:] ) )
    for i in range(n):
        df[:, i] = np.exp( -r * 1/n * (i+1) )
    
    # calculate the present value of each path
    PV_of_Cash_Flow = np.sum( (Cash_Flow[:, 1:] * df), axis=1 )
    
    # calculate the value of the option
    # by averaging the value of each presen value of each path
    value = np.mean(PV_of_Cash_Flow)
    
    return value



In [277]:
S0 = 36
K = 40
r = 0.06
sigma = 0.4
t = 0
T = 1
d = 0
n = 5 # 50 steps per year
m = 10 # 100000 paths of stock price

In [278]:
Value_LSM = Valuation_by_Least_Square( r, sigma, S0, m, n, t, T )
Value_LSM

4.520956796726551

In [279]:
St_GeoBro = np.zeros((m, n+1))
for i in range(m):
    St_GeoBro[i, :] = Geometric_Brownian_Motion_Trajectory( r, sigma, S0, n, t, T )

In [280]:
st = pd.DataFrame(data=St_GeoBro)
st

,0,1,2,3,4,5
0,36.0,30.886417,50.486872,49.621904,44.419210,33.168587
1,36.0,28.122327,27.023809,24.240287,18.859292,20.430142
2,36.0,32.513081,38.820187,42.573805,47.182419,43.525131
3,36.0,36.149456,29.336596,27.606118,26.783914,39.239713
4,36.0,33.494752,41.255563,43.171810,50.842742,47.945880
5,36.0,34.535154,33.872100,39.951350,46.838092,65.453322
6,36.0,40.000867,45.992322,45.318806,43.776572,81.039723
7,36.0,35.413704,26.709763,23.792019,28.292860,24.177192
8,36.0,34.987295,32.262497,38.769914,34.910185,42.533400
9,36.0,37.217069,43.258585,52.181505,48.209809,52.454973


In [281]:
Payoff = np.maximum( K - St_GeoBro, 0 )
Payoff_df = pd.DataFrame(data=Payoff)

Cash_Flow = np.maximum( K - St_GeoBro, 0 )
Cash_Flow_df = pd.DataFrame(data=Cash_Flow)

In [282]:
Cash_Flow_df

,0,1,2,3,4,5
0,4.0,9.113583,0.000000,0.000000,0.000000,6.831413
1,4.0,11.877673,12.976191,15.759713,21.140708,19.569858
2,4.0,7.486919,1.179813,0.000000,0.000000,0.000000
3,4.0,3.850544,10.663404,12.393882,13.216086,0.760287
4,4.0,6.505248,0.000000,0.000000,0.000000,0.000000
5,4.0,5.464846,6.127900,0.048650,0.000000,0.000000
6,4.0,0.000000,0.000000,0.000000,0.000000,0.000000
7,4.0,4.586296,13.290237,16.207981,11.707140,15.822808
8,4.0,5.012705,7.737503,1.230086,5.089815,0.000000
9,4.0,2.782931,0.000000,0.000000,0.000000,0.000000


In [283]:
Payoff_df

,0,1,2,3,4,5
0,4.0,9.113583,0.000000,0.000000,0.000000,6.831413
1,4.0,11.877673,12.976191,15.759713,21.140708,19.569858
2,4.0,7.486919,1.179813,0.000000,0.000000,0.000000
3,4.0,3.850544,10.663404,12.393882,13.216086,0.760287
4,4.0,6.505248,0.000000,0.000000,0.000000,0.000000
5,4.0,5.464846,6.127900,0.048650,0.000000,0.000000
6,4.0,0.000000,0.000000,0.000000,0.000000,0.000000
7,4.0,4.586296,13.290237,16.207981,11.707140,15.822808
8,4.0,5.012705,7.737503,1.230086,5.089815,0.000000
9,4.0,2.782931,0.000000,0.000000,0.000000,0.000000


In [284]:
i = 0
X = ( Payoff[:, n-1-i] )[ Payoff[:, n-1-i] > 0 ]
X

array([21.14070805, 13.21608576, 11.70713977,  5.08981508])

In [285]:
Y = ( Payoff[:, n-i] )[ Payoff[:, n-1-i] > 0 ] 
Y

array([19.56985794,  0.76028744, 15.82280816,  0.        ])

In [286]:
Y = ( Payoff[:, n-i] )[ Payoff[:, n-1-i] > 0 ] * np.exp( -r * 1/n * (i+1) )
Y

array([19.33642305,  0.75121851, 15.63406916,  0.        ])

In [287]:
X = X.reshape(np.size(X),1)
X

array([[21.14070805],
       [13.21608576],
       [11.70713977],
       [ 5.08981508]])

In [288]:
L0 = np.exp( -X/2 )
L1 = L0 * ( 1 - X )
L2 = L0 * ( 1 - 2*X + X**2/2 )

In [289]:
L0

array([[2.56657273e-05],
       [1.34947064e-03],
       [2.86963657e-03],
       [7.84803081e-02]])

In [290]:
L1

array([[-0.00051693],
       [-0.01648525],
       [-0.0307256 ],
       [-0.32096995]])

In [291]:
L2

array([[0.00467587],
       [0.08353262],
       [0.13233123],
       [0.29614376]])

In [292]:
XX = np.hstack((L0, L1, L2))
XX

array([[ 2.56657273e-05, -5.16925920e-04,  4.67586823e-03],
       [ 1.34947064e-03, -1.64852491e-02,  8.35326240e-02],
       [ 2.86963657e-03, -3.07255999e-02,  1.32331228e-01],
       [ 7.84803081e-02, -3.20969947e-01,  2.96143763e-01]])

In [293]:
Y

array([19.33642305,  0.75121851, 15.63406916,  0.        ])

In [294]:
Y_predict = reg.predict(XX)
Y_predict

array([[ 9.81596471],
       [10.95158227],
       [11.54417366],
       [ 8.18017129]])

In [295]:
Payoff[ Payoff[:, n-1-i] > 0, n-1-i ]

array([21.14070805, 13.21608576, 11.70713977,  5.08981508])

In [296]:
Y_predict = Y_predict.reshape(1,np.size(Y_predict))
Y_predict

array([[ 9.81596471, 10.95158227, 11.54417366,  8.18017129]])

In [297]:
exercise = ( Y_predict < Payoff[ Payoff[:, n-1-i] > 0, n-1-i ] )
exercise

array([[ True,  True,  True, False]])

In [298]:
exercise = ( Y_predict < Payoff[ Payoff[:, n-1-i] > 0, n-1-i ] ) * Payoff[ Payoff[:, n-1-i] > 0, n-1-i ]
exercise

array([[21.14070805, 13.21608576, 11.70713977,  0.        ]])

In [299]:
Cash_Flow[ Payoff[:, n-1-i] > 0, n-1-i ] = exercise
Cash_Flow[ Payoff[:, n-1-i] > 0, n-i: ] = 0

In [300]:
Cash_Flow_df = pd.DataFrame(data=Cash_Flow)
Cash_Flow_df

,0,1,2,3,4,5
0,4.0,9.113583,0.000000,0.000000,0.000000,6.831413
1,4.0,11.877673,12.976191,15.759713,21.140708,0.000000
2,4.0,7.486919,1.179813,0.000000,0.000000,0.000000
3,4.0,3.850544,10.663404,12.393882,13.216086,0.000000
4,4.0,6.505248,0.000000,0.000000,0.000000,0.000000
5,4.0,5.464846,6.127900,0.048650,0.000000,0.000000
6,4.0,0.000000,0.000000,0.000000,0.000000,0.000000
7,4.0,4.586296,13.290237,16.207981,11.707140,0.000000
8,4.0,5.012705,7.737503,1.230086,0.000000,0.000000
9,4.0,2.782931,0.000000,0.000000,0.000000,0.000000


In [306]:
d = np.zeros((5,5))
test = pd.DataFrame(data=d)
test

,0,1,2,3,4
0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0


In [339]:
d

array([[0., 1., 2., 0., 0.],
       [0., 1., 2., 0., 0.],
       [0., 1., 2., 0., 0.],
       [0., 1., 2., 0., 0.],
       [0., 1., 2., 0., 0.]])

In [340]:
d[:, 2] = 2
d[:, 1] = 1
d

array([[0., 1., 2., 0., 0.],
       [0., 1., 2., 0., 0.],
       [0., 1., 2., 0., 0.],
       [0., 1., 2., 0., 0.],
       [0., 1., 2., 0., 0.]])

In [341]:
x = (d[:, 1])[ d[:, 2] > 0 ]
x

array([1., 1., 1., 1., 1.])

In [342]:
x.size

5

In [343]:
d[:, 4] > 0 

array([False, False, False, False, False])

In [344]:
x = (d[:, 1])[ d[:, 4] > 0 ]
x

array([], dtype=float64)

In [347]:
if x.size==0:
    print(1)

1
